In [32]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

In [2]:
import gym_tetris
import random
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT

In [3]:
# class JoypadSpaceMixin(JoypadSpace):
#     def step(self, action):
#         n_state, reward, done, info = super().step(action)
#         return n_state, reward, done, info

In [4]:
env = gym_tetris.make('TetrisA-v0')
env = JoypadSpace(env, MOVEMENT)
states = env.observation_space.shape[0]
actions = env.action_space.n
states, actions

(240, 12)

In [5]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        n_state, reward, done, info = env.step(env.action_space.sample())
        score+=reward
        print("info:")
        print(info)
        print()
    print('Episode:{} Score:{}'.format(episode, score))

info:
{'current_piece': 'O', 'number_of_lines': 0, 'score': 0, 'next_piece': 'O', 'statistics': {'T': 0, 'J': 0, 'Z': 0, 'O': 1, 'S': 0, 'L': 0, 'I': 0}, 'board_height': 0}

info:
{'current_piece': 'O', 'number_of_lines': 0, 'score': 0, 'next_piece': 'O', 'statistics': {'T': 0, 'J': 0, 'Z': 0, 'O': 1, 'S': 0, 'L': 0, 'I': 0}, 'board_height': 0}

info:
{'current_piece': 'O', 'number_of_lines': 0, 'score': 0, 'next_piece': 'O', 'statistics': {'T': 0, 'J': 0, 'Z': 0, 'O': 1, 'S': 0, 'L': 0, 'I': 0}, 'board_height': 0}

info:
{'current_piece': 'O', 'number_of_lines': 0, 'score': 0, 'next_piece': 'O', 'statistics': {'T': 0, 'J': 0, 'Z': 0, 'O': 1, 'S': 0, 'L': 0, 'I': 0}, 'board_height': 0}

info:
{'current_piece': 'O', 'number_of_lines': 0, 'score': 0, 'next_piece': 'O', 'statistics': {'T': 0, 'J': 0, 'Z': 0, 'O': 1, 'S': 0, 'L': 0, 'I': 0}, 'board_height': 0}

info:
{'current_piece': 'O', 'number_of_lines': 0, 'score': 0, 'next_piece': 'O', 'statistics': {'T': 0, 'J': 0, 'Z': 0, 'O': 1, '

KeyboardInterrupt: 

In [60]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [61]:
def build_model(states, actions):
    model = tensorflow.keras.models.Sequential()
    model.add(Flatten(input_shape=(1, 240, 256, 3)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [62]:
model = build_model(states, actions)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 184320)            0         
                                                                 
 dense_15 (Dense)            (None, 24)                4423704   
                                                                 
 dense_16 (Dense)            (None, 24)                600       
                                                                 
 dense_17 (Dense)            (None, 12)                300       
                                                                 
Total params: 4,424,604
Trainable params: 4,424,604
Non-trainable params: 0
_________________________________________________________________


In [63]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [64]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [65]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 6132s 613ms/step - reward: 0.0000e+00
Interval 2 (10000 steps performed)
10000/10000 [==============================] - 5933s 593ms/step - reward: 0.0000e+00
1 episodes - episode_reward: 0.000 [0.000, 0.000] - loss: 26478.633 - mae: 3.258 - mean_q: 4.990 - number_of_lines: 0.000 - score: 0.000 - board_height: 10.376

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 5950s 595ms/step - reward: 1.0000e-04
2 episodes - episode_reward: 0.500 [0.000, 1.000] - loss: 0.000 - mae: 1.184 - mean_q: 1.305 - number_of_lines: 0.000 - score: 0.625 - board_height: 8.066

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 5877s 588ms/step - reward: 0.0000e+00
1 episodes - episode_reward: 0.000 [0.000, 0.000] - loss: 0.000 - mae: 0.648 - mean_q: 0.708 - number_of_lines: 0.000 - score: 0.000 - board_height: 7.598

Interval 5 (40000 step

In [66]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 5.000, steps: 4421
Episode 2: reward: 24.000, steps: 3723
Episode 3: reward: 4.000, steps: 4367
Episode 4: reward: 1.000, steps: 4690
Episode 5: reward: 6.000, steps: 4647
Episode 6: reward: 19.000, steps: 3643
Episode 7: reward: 0.000, steps: 5379
Episode 8: reward: 20.000, steps: 4219
Episode 9: reward: 16.000, steps: 3492
Episode 10: reward: 9.000, steps: 2572
Episode 11: reward: 8.000, steps: 4053
Episode 12: reward: 14.000, steps: 4929
Episode 13: reward: 12.000, steps: 4008
Episode 14: reward: 31.000, steps: 2133
Episode 15: reward: 10.000, steps: 4435
Episode 16: reward: 7.000, steps: 4743
Episode 17: reward: 10.000, steps: 4264
Episode 18: reward: 26.000, steps: 2644
Episode 19: reward: 25.000, steps: 2883
Episode 20: reward: 1.000, steps: 5192
Episode 21: reward: 0.000, steps: 5736
Episode 22: reward: 13.000, steps: 4220
Episode 23: reward: 10.000, steps: 2527
Episode 24: reward: 4.000, steps: 4551
Episode 25: reward: 1.000, step

In [1]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

NameError: name 'dqn' is not defined

Среднее число шагов до проигрыша:

In [9]:
s = """Episode 1: reward: 5.000, steps: 4421
Episode 2: reward: 24.000, steps: 3723
Episode 3: reward: 4.000, steps: 4367
Episode 4: reward: 1.000, steps: 4690
Episode 5: reward: 6.000, steps: 4647
Episode 6: reward: 19.000, steps: 3643
Episode 7: reward: 0.000, steps: 5379
Episode 8: reward: 20.000, steps: 4219
Episode 9: reward: 16.000, steps: 3492
Episode 10: reward: 9.000, steps: 2572
Episode 11: reward: 8.000, steps: 4053
Episode 12: reward: 14.000, steps: 4929
Episode 13: reward: 12.000, steps: 4008
Episode 14: reward: 31.000, steps: 2133
Episode 15: reward: 10.000, steps: 4435
Episode 16: reward: 7.000, steps: 4743
Episode 17: reward: 10.000, steps: 4264
Episode 18: reward: 26.000, steps: 2644
Episode 19: reward: 25.000, steps: 2883
Episode 20: reward: 1.000, steps: 5192
Episode 21: reward: 0.000, steps: 5736
Episode 22: reward: 13.000, steps: 4220
Episode 23: reward: 10.000, steps: 2527
Episode 24: reward: 4.000, steps: 4551
Episode 25: reward: 1.000, steps: 5644
Episode 26: reward: 18.000, steps: 3510
Episode 27: reward: 2.000, steps: 4879
Episode 28: reward: 0.000, steps: 5330
Episode 29: reward: 1.000, steps: 6111
Episode 30: reward: 14.000, steps: 3519
Episode 31: reward: 23.000, steps: 3379
Episode 32: reward: 0.000, steps: 4731
Episode 33: reward: 2.000, steps: 5315
Episode 34: reward: 3.000, steps: 5498
Episode 35: reward: 18.000, steps: 3164
Episode 36: reward: 14.000, steps: 4768
Episode 37: reward: 1.000, steps: 3803
Episode 38: reward: 1.000, steps: 4960
Episode 39: reward: 12.000, steps: 3710
Episode 40: reward: 0.000, steps: 4833
Episode 41: reward: 4.000, steps: 4488
Episode 42: reward: 15.000, steps: 3928
Episode 43: reward: 1.000, steps: 5880
Episode 44: reward: 3.000, steps: 5400
Episode 45: reward: 10.000, steps: 4545
Episode 46: reward: 16.000, steps: 4454
Episode 47: reward: 22.000, steps: 3383
Episode 48: reward: 19.000, steps: 3369
Episode 49: reward: 8.000, steps: 4100
Episode 50: reward: 10.000, steps: 4849
Episode 51: reward: 12.000, steps: 4536
Episode 52: reward: 0.000, steps: 5688
Episode 53: reward: 15.000, steps: 4409
Episode 54: reward: 1.000, steps: 5550
Episode 55: reward: 31.000, steps: 2503
Episode 56: reward: 3.000, steps: 5767
Episode 57: reward: 31.000, steps: 3827
Episode 58: reward: 11.000, steps: 4275
Episode 59: reward: 13.000, steps: 5012
Episode 60: reward: 12.000, steps: 4202
Episode 61: reward: 11.000, steps: 3861
Episode 62: reward: 15.000, steps: 4386
Episode 63: reward: 0.000, steps: 4993
Episode 64: reward: 19.000, steps: 2548
Episode 65: reward: 20.000, steps: 3127
Episode 66: reward: 0.000, steps: 5453
Episode 67: reward: 0.000, steps: 5404
Episode 68: reward: 5.000, steps: 5239
Episode 69: reward: 5.000, steps: 4994
Episode 70: reward: 10.000, steps: 4257
Episode 71: reward: 5.000, steps: 4921
Episode 72: reward: 25.000, steps: 3447
Episode 73: reward: 7.000, steps: 4878
Episode 74: reward: 31.000, steps: 2850
Episode 75: reward: 4.000, steps: 4313
Episode 76: reward: 26.000, steps: 3574
Episode 77: reward: 9.000, steps: 3241
Episode 78: reward: 25.000, steps: 3584
Episode 79: reward: 14.000, steps: 3566
Episode 80: reward: 2.000, steps: 4411
Episode 81: reward: 9.000, steps: 4385
Episode 82: reward: 10.000, steps: 4991
Episode 83: reward: 0.000, steps: 4847
Episode 84: reward: 1.000, steps: 5475
Episode 85: reward: 15.000, steps: 3035
Episode 86: reward: 12.000, steps: 4370
Episode 87: reward: 15.000, steps: 3496
Episode 88: reward: 3.000, steps: 4432
Episode 89: reward: 5.000, steps: 4508
Episode 90: reward: 11.000, steps: 4529
Episode 91: reward: 8.000, steps: 4307
Episode 92: reward: 24.000, steps: 2885
Episode 93: reward: 10.000, steps: 4088
Episode 94: reward: 13.000, steps: 3485
Episode 95: reward: 21.000, steps: 2657
Episode 96: reward: 35.000, steps: 3852
Episode 97: reward: 26.000, steps: 3480
Episode 98: reward: 0.000, steps: 5493
Episode 99: reward: 10.000, steps: 4529
Episode 100: reward: 19.000, steps: 3342"""
sum([int(line.split()[-1]) for line in s.split('\n')])/len(s.split('\n'))

4260.23